# <center> 🎰 Automatic feature generaton </center>

Как говорил Конфуций: ***"Зачем делать что-то самому, если можно заставить сделать это самое что-то кого-то другого?"***

<p><font size="3" face="Arial" font-size="large"><ul type="square">
    
<li><a href="#c1">🧰 Featuretools - для данных в виде sql-базы. </a></li>
<li><a href="#c2">🌐 GeoPandas - для работы с геоданными. </a></li>
<li><a href="#с3">🥋 Karateclub и NetworkX - для графов.</a>
<li><a href="#c4"> 🥒 Tsfresh - для временных рядов.</a>
<li><a href="#6">🧸 Выводы и заключения.</a>

<center> <img src='https://i.pinimg.com/474x/25/1b/63/251b631e41794b934f4632a28a2c5127.jpg' width="400" height="400" /> </center> 

</li></ul></font></p>

In [1]:
import pandas as pd
import featuretools as ft
from woodwork.logical_types import Categorical, Double, Age, Datetime

In [2]:
path = '../../data/car_train.csv'
car_info = pd.read_csv(path)

path = '../../data/driver_info.csv'
driver_info = pd.read_csv(path)

path = '../../data/rides_info.csv'
rides_info = pd.read_csv(path)

path = '../../data/fix_info.csv'
fix_info = pd.read_csv(path)

In [3]:
es = ft.EntitySet(id="car_data")

In [4]:
es = es.add_dataframe(
    dataframe_name="cars",
    dataframe=car_info,
    index="car_id",
    logical_types={"car_type": Categorical, 'fuel_type': Categorical, 'model': Categorical}
    )

es = es.add_dataframe(
    dataframe_name="rides",
    dataframe=rides_info.drop(['ride_id'], axis=1),
    index='index',
    time_index="ride_date",
    )

es = es.add_dataframe(
    dataframe_name="drivers",
    dataframe=driver_info,
    index="user_id",
    logical_types={"sex": Categorical, "first_ride_date": Datetime, "age": Age}
    )

es = es.add_dataframe(
    dataframe_name="fixes",
    dataframe=fix_info,
    index="index",
    logical_types={"work_type": Categorical, "worker_id":Categorical}
    )
es

/home/artem/projects/ml_learning/comp_ds/_build/pip_packages/lib/python3.10/site-packages/featuretools/entityset/entityset.py:1910: UserWarning: index index not found in dataframe, creating new integer column
  warnings.warn(
/home/artem/projects/ml_learning/comp_ds/_build/pip_packages/lib/python3.10/site-packages/woodwork/logical_types.py:385: TypeConversionWarning: Some rows in series 'first_ride_date' are incompatible with datetime format '%Y-%m-%d' and have been replaced with null values. You may be able to fix this by using an instantiated Datetime logical type with a different format string specified for this column during Woodwork initialization.
  warnings.warn(
/home/artem/projects/ml_learning/comp_ds/_build/pip_packages/lib/python3.10/site-packages/featuretools/entityset/entityset.py:1910: UserWarning: index index not found in dataframe, creating new integer column
  warnings.warn(


Entityset: car_data
  DataFrames:
    cars [Rows: 2337, Columns: 10]
    rides [Rows: 739500, Columns: 14]
    drivers [Rows: 15153, Columns: 7]
    fixes [Rows: 146000, Columns: 7]
  Relationships:
    No relationships

In [5]:
es = es.add_relationship("cars", "car_id", "rides", "car_id")
es = es.add_relationship("drivers", "user_id", "rides", "user_id")
es = es.add_relationship("cars", "car_id", "fixes", "car_id")

es

/home/artem/projects/ml_learning/comp_ds/_build/pip_packages/lib/python3.10/site-packages/featuretools/entityset/entityset.py:414: UserWarning: Logical type Categorical for child column car_id does not match parent column car_id logical type Unknown. Changing child logical type to match parent.
  warnings.warn(
/home/artem/projects/ml_learning/comp_ds/_build/pip_packages/lib/python3.10/site-packages/featuretools/entityset/entityset.py:414: UserWarning: Logical type Categorical for child column user_id does not match parent column user_id logical type Unknown. Changing child logical type to match parent.
  warnings.warn(


Entityset: car_data
  DataFrames:
    cars [Rows: 2337, Columns: 10]
    rides [Rows: 739500, Columns: 14]
    drivers [Rows: 15153, Columns: 7]
    fixes [Rows: 146000, Columns: 7]
  Relationships:
    rides.car_id -> cars.car_id
    rides.user_id -> drivers.user_id
    fixes.car_id -> cars.car_id

In [6]:
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="cars",
    max_depth=1,
)
feature_matrix.head()

,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class,COUNT(rides),...,MODE(fixes.work_type),MODE(fixes.worker_id),NUM_UNIQUE(fixes.work_type),NUM_UNIQUE(fixes.worker_id),SKEW(fixes.destroy_degree),SKEW(fixes.work_duration),STD(fixes.destroy_degree),STD(fixes.work_duration),SUM(fixes.destroy_degree),SUM(fixes.work_duration)
car_id,,,,,,,,,,,,,,,,,,,,,
y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,9492.96,another_bug,174,...,reparking,LR,4,33,0.835907,0.826462,2.732847,10.171884,106.7,933.0
O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,2656.23,electro_bug,174,...,reparking,YH,5,34,0.997276,-0.296841,2.707233,8.574733,102.1,873.0
d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,1526.11,gear_stick,174,...,repair,AP,5,35,0.472628,0.671481,2.978077,13.040983,130.9,915.0
u29695600e,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,1338.00,engine_fuel,174,...,repair,LM,4,34,0.492743,0.639490,3.237750,14.764994,143.0,1007.0
N-8915870N,Renault Sandero,standart,petrol,4.70,2012,26428,2017,825.72,engine_fuel,174,...,repair,CD,4,34,0.478043,1.341642,3.216758,12.659537,135.8,981.0


In [7]:
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="cars",
    agg_primitives=["mode", "count"], # Ограничиваем число фичей
    max_depth=1, # Ограничиваем глубину
)
feature_matrix.head()

,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class,COUNT(rides),COUNT(fixes),MODE(fixes.work_type),MODE(fixes.worker_id)
car_id,,,,,,,,,,,,,
y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,9492.96,another_bug,174,35,reparking,LR
O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,2656.23,electro_bug,174,35,reparking,YH
d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,1526.11,gear_stick,174,35,repair,AP
u29695600e,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,1338.00,engine_fuel,174,35,repair,LM
N-8915870N,Renault Sandero,standart,petrol,4.70,2012,26428,2017,825.72,engine_fuel,174,35,repair,CD


HomeWork

In [8]:
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="cars",
    agg_primitives=["sum", "count", "max"],
    instance_ids=list(car_info[car_info['model'] == 'Tesla Model 3']['car_id']),
    max_depth=2,
)

In [21]:
feature_matrix['SUM(fixes.work_duration)'].max()

970.0